In [1]:
# setup spark for processing transactions

import findspark
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

findspark.init()
spark_conf = pyspark.SparkConf()\
    .setMaster('local[*]')\
    .set('spark.local.dir', '/scratch/spark')\
    .set('spark.driver.memory', '100g')\
    .setAppName('eth')

spark_context = SparkContext(conf=spark_conf)
spark_session = SparkSession(spark_context)

In [2]:
from src.EthTransactions import EthTransactions, TimeSpan, DataFrameFormat

# load transdactions exported from ethereum-etl
eth_transactions = EthTransactions(spark_session, '/ssd_raid/eth/etl/transactions.csv')

# process transactions per day
eth_transactions.top_transactions(500, TimeSpan.DAY, 'data/transactions/top_daily', DataFrameFormat.PARQUET)
eth_transactions.stats(TimeSpan.DAY, 'data/transactions/stats_daily', DataFrameFormat.PARQUET)

# process transactions per hour
eth_transactions.top_transactions(500, TimeSpan.HOUR, 'data/transactions/top_hourly', DataFrameFormat.PARQUET)
eth_transactions.stats(TimeSpan.HOUR, 'data/transactions/stats_hourly', DataFrameFormat.PARQUET)